In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import sys
import h5py
import numpy as np
import matplotlib.pyplot as plt

In [34]:
PROJECT_PATH = '../'
DATA_PATH = '/scratch/ceph/swei20/data/ae/dataset/test'
PCA_PATH = '/scratch/ceph/dobos/data/pfsspec/import/stellar/rbf/bosz_5000_full/pca/spectra.h5'
sys.path.insert(0, PROJECT_PATH)

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
import tensorflow as tf
# tf.enable_v2_behavior()
gpus = tf.config.list_physical_devices('GPU') 
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Loading Data

In [35]:
with h5py.File(PCA_PATH, 'r') as f:
    flux = f['flux'][()]

In [39]:
def prepro_pca(flux, dim=300):
    flux0 = flux.reshape(-1, 1000)[:, :dim]    
    flux_clean = flux0[~np.isnan(flux0[:,0])]    
    print(flux_clean.shape, flux0.shape)
    print('min', flux_clean.min(), 'max', flux_clean.max(),'mean', flux_clean.mean(),'std', flux_clean.std() )
    return flux_clean

In [40]:
flux1 = prepro_pca(flux, dim=300)

(116614, 300) (243936, 300)
min -184.9254058126797 max 145.24091665337806 mean -0.10353327716635463 std 2.4000601615265493


In [27]:
with h5py.File(fn, 'r') as f:
    flux10kmm = f['flux10kmm'][()]
#     flux10kmm = f['flux10kmm'][()]

## AE

In [84]:
from dotmap import DotMap

In [85]:
from ae.model.simple_ae_model import SimpleAEModel

In [86]:
config_dict = {
    "data": {
      "dir": "/scratch/ceph/swei20/data/ae/dataset/test",
      "filename": "tt.h5",
      "train": "flux10kmm",
      "test": "flux10kmm"
    },
    "model":{
      "name": "SimpleAE",
      "input_dim": 300,
      "latent_dim": 8,
      "hidden_dims": [],
      "dropout": 0.2,
      "lr": 0.001,
      "reg1": 0.0,
      "loss": "mae",
      "opt": "adam"
    },
    "trainer":{
      "epoch": 20,
      "batch_size": 32,
      "validation_split":0.1,
      "verbose_training": 1
    },
    "callbacks":{
        "checkpoint_monitor": "val_loss",
        "checkpoint_mode": "min"
    }
}

In [87]:
config = DotMap(config_dict)

In [88]:
mm = SimpleAEModel(config)

[300, 8]


In [89]:
mm.build_model()

In [90]:
from ae.trainer.simple_ae_trainer import SimpleAETrainer

In [91]:
tt = SimpleAETrainer(mm, flux1,config)

In [92]:
history = tt.train()

Epoch 1/20
3280/3280 [==============================] - 6s 2ms/step - loss: 0.1557 - acc: 0.4236 - val_loss: 0.1724 - val_acc: 0.2992
Epoch 2/20
3280/3280 [==============================] - 5s 2ms/step - loss: 0.1188 - acc: 0.5527 - val_loss: 0.1693 - val_acc: 0.3395
Epoch 3/20
3280/3280 [==============================] - 5s 2ms/step - loss: 0.1148 - acc: 0.5995 - val_loss: 0.1665 - val_acc: 0.4565
Epoch 4/20
3280/3280 [==============================] - 5s 2ms/step - loss: 0.1130 - acc: 0.6182 - val_loss: 0.1652 - val_acc: 0.4713
Epoch 5/20
3280/3280 [==============================] - 5s 2ms/step - loss: 0.1122 - acc: 0.6215 - val_loss: 0.1659 - val_acc: 0.4711
Epoch 6/20
3280/3280 [==============================] - 5s 2ms/step - loss: 0.1121 - acc: 0.6242 - val_loss: 0.1657 - val_acc: 0.4739
Epoch 7/20
3280/3280 [==============================] - 5s 2ms/step - loss: 0.1115 - acc: 0.6284 - val_loss: 0.1656 - val_acc: 0.5002
Epoch 8/20
3280/3280 [==============================] - 5s 2ms